In [72]:
import pandas as pd

In [73]:
transaction = pd.read_csv('transactions.csv')

In [75]:
display(transaction)

,account_number,transaction_datetime,transaction_amount,post_date,merchant_number,merchant_description,merchant_category_code,transaction_number
0,26522,12182020 15:29:12,1.06-,12192020,000000000326552,MAVERIK #488 DRAPER LAUS,5541,3
1,26522,12302020 08:22:36,1.72-,12312020,900185000000000,HARMONS - BANGERTER DRAPER LA,5411,8
2,11111,02192021 09:55:25,1009.90-,2202021,527021002118976,CFA Institute 4349515499 TX,8299,1
3,11111,02092021 11:15:52,14.00-,2112021,000225124800999,CHAPULTEPEC LUPITA HOUSTON TX,5812,2
4,26522,01272021 09:34:44,0.90-,1272021,650000010604109,PCBC SPRINGVILLE SPRINGVILLE LA,5814,2
...,...,...,...,...,...,...,...,...
2517,37222,02272021 15:27:57,4.85-,3012021,000000002355702,TIGER EXPRESS Los Angeles CA,5499,2
2518,37222,04012021 06:17:18,50.04-,4022021,053894000053360,SHELL OIL Los Angeles CA,5814,3
2519,37222,03122021 07:33:49,45.73-,3132021,4445025811745,SHELL OIL\t\tLos Angeles CA,5812,4
2520,37222,01212021 12:28:59,125.00-,1232021,000021668538000,SHELL OIL Los Angeles CA,5812,5


In [76]:
transaction['merchant_name'] = transaction['merchant_description'].map(lambda x: x.split(" ")[0])
transaction['merchant_state']= transaction['merchant_description'].map(lambda x: x.split(" ")[-1])

In [77]:
display(transaction)

,account_number,transaction_datetime,transaction_amount,post_date,merchant_number,merchant_description,merchant_category_code,transaction_number,merchant_name,merchant_state
0,26522,12182020 15:29:12,1.06-,12192020,000000000326552,MAVERIK #488 DRAPER LAUS,5541,3,MAVERIK,LAUS
1,26522,12302020 08:22:36,1.72-,12312020,900185000000000,HARMONS - BANGERTER DRAPER LA,5411,8,HARMONS,LA
2,11111,02192021 09:55:25,1009.90-,2202021,527021002118976,CFA Institute 4349515499 TX,8299,1,CFA,TX
3,11111,02092021 11:15:52,14.00-,2112021,000225124800999,CHAPULTEPEC LUPITA HOUSTON TX,5812,2,CHAPULTEPEC,TX
4,26522,01272021 09:34:44,0.90-,1272021,650000010604109,PCBC SPRINGVILLE SPRINGVILLE LA,5814,2,PCBC,LA
...,...,...,...,...,...,...,...,...,...,...
2517,37222,02272021 15:27:57,4.85-,3012021,000000002355702,TIGER EXPRESS Los Angeles CA,5499,2,TIGER,CA
2518,37222,04012021 06:17:18,50.04-,4022021,053894000053360,SHELL OIL Los Angeles CA,5814,3,SHELL,CA
2519,37222,03122021 07:33:49,45.73-,3132021,4445025811745,SHELL OIL\t\tLos Angeles CA,5812,4,SHELL,CA
2520,37222,01212021 12:28:59,125.00-,1232021,000021668538000,SHELL OIL Los Angeles CA,5812,5,SHELL,CA


In [78]:
states = [ 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
           'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
           'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
           'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
           'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']

In [79]:
transaction['merchant_state'] = transaction['merchant_state'].map(lambda x: x[0:2] )

In [80]:
display(transaction)

,account_number,transaction_datetime,transaction_amount,post_date,merchant_number,merchant_description,merchant_category_code,transaction_number,merchant_name,merchant_state
0,26522,12182020 15:29:12,1.06-,12192020,000000000326552,MAVERIK #488 DRAPER LAUS,5541,3,MAVERIK,LA
1,26522,12302020 08:22:36,1.72-,12312020,900185000000000,HARMONS - BANGERTER DRAPER LA,5411,8,HARMONS,LA
2,11111,02192021 09:55:25,1009.90-,2202021,527021002118976,CFA Institute 4349515499 TX,8299,1,CFA,TX
3,11111,02092021 11:15:52,14.00-,2112021,000225124800999,CHAPULTEPEC LUPITA HOUSTON TX,5812,2,CHAPULTEPEC,TX
4,26522,01272021 09:34:44,0.90-,1272021,650000010604109,PCBC SPRINGVILLE SPRINGVILLE LA,5814,2,PCBC,LA
...,...,...,...,...,...,...,...,...,...,...
2517,37222,02272021 15:27:57,4.85-,3012021,000000002355702,TIGER EXPRESS Los Angeles CA,5499,2,TIGER,CA
2518,37222,04012021 06:17:18,50.04-,4022021,053894000053360,SHELL OIL Los Angeles CA,5814,3,SHELL,CA
2519,37222,03122021 07:33:49,45.73-,3132021,4445025811745,SHELL OIL\t\tLos Angeles CA,5812,4,SHELL,CA
2520,37222,01212021 12:28:59,125.00-,1232021,000021668538000,SHELL OIL Los Angeles CA,5812,5,SHELL,CA


In [81]:
del(transaction['merchant_description'])

In [82]:
transaction

,account_number,transaction_datetime,transaction_amount,post_date,merchant_number,merchant_category_code,transaction_number,merchant_name,merchant_state
0,26522,12182020 15:29:12,1.06-,12192020,000000000326552,5541,3,MAVERIK,LA
1,26522,12302020 08:22:36,1.72-,12312020,900185000000000,5411,8,HARMONS,LA
2,11111,02192021 09:55:25,1009.90-,2202021,527021002118976,8299,1,CFA,TX
3,11111,02092021 11:15:52,14.00-,2112021,000225124800999,5812,2,CHAPULTEPEC,TX
4,26522,01272021 09:34:44,0.90-,1272021,650000010604109,5814,2,PCBC,LA
...,...,...,...,...,...,...,...,...,...
2517,37222,02272021 15:27:57,4.85-,3012021,000000002355702,5499,2,TIGER,CA
2518,37222,04012021 06:17:18,50.04-,4022021,053894000053360,5814,3,SHELL,CA
2519,37222,03122021 07:33:49,45.73-,3132021,4445025811745,5812,4,SHELL,CA
2520,37222,01212021 12:28:59,125.00-,1232021,000021668538000,5812,5,SHELL,CA


In [83]:
transaction.to_csv('transaction_cleaned.csv', index=False)  